## Scraping the twitter dataset

### testing

#### Testing different api requests for twitter
#api - user_timeline
#api - home_timeline
#api - friends

In [ ]:
def process_or_store(tweet):
    print(json.dumps(tweet))

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    process_or_store(status._json)

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status._json)

In [ ]:
## getting list of users I am following (api - friends)
# GET/request endpoint of twitter
for friend in tweepy.Cursor(api.friends).items():
    print(friend._json['name'])

In [ ]:
#printing tweets (api - home_timeline)
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.created_at)

In [ ]:
## tweets in the user time line (api - user_timeline)
for tweet in tweepy.Cursor(api.user_timeline).items():
    process_or_store(tweet._json)

In [ ]:
#getting all the juulvapor tweets and creating a csv file
def generic_info():
    columns = ['id','created_at','text','tweet_hashtags','tweet_retweetCount','likes','user','user_friends',]
    df = pd.concat([pd.DataFrame(
       [(tweet.id,
         tweet.created_at,
         tweet.text,
         [element['text'] for element in tweet.entities['hashtags']],
         tweet.retweet_count,
         tweet.favorite_count,
         tweet.user.screen_name,
         tweet.user.friends,
        )]
        ,columns=columns) 
            for tweet in tweepy.Cursor(api.search,q=('#juulvapor OR #juulnation OR #doit4juul OR #juul'),lang="en",since_id="2018-01-01").items()],
                ignore_index=True)
    return df

In [ ]:
#2 sep 2018
# getting all the user info in the twitter
#getting all the retweet counts
# need to change the inception ID

def userInfo():
    columns = ['tweet_id','user_id','user.name','user_desc','user_followerCount','user_friendsCount',
           'userCreatedAt','userFavriteCount','userProfileImageUrl','userFriendslist']
    df_test = pd.concat(
        [pd.DataFrame(
        [(tweet.id,
          tweet.user.id,
          tweet.user.name,
          tweet.user.description,
          tweet.user.followers_count,
          tweet.user.friends_count,
          tweet.user.created_at,
          tweet.user.favourites_count,
          tweet.user.profile_image_url,
          [friend for friend in tweepy.Cursor(api.friends_ids, id= tweet.user.id, lang="en",since_id= "2018-08-08").items(6)]
         )]
         ,columns=columns) 
            for tweet in tweepy.Cursor(api.search,q=('#juulvapor OR #juulnation OR #doit4juul OR #juul'),lang="en",since_id="2018-01-01").items(1)],
                ignore_index=True)
    return(df_test)

## Creating a different table for friends list and their description

In [ ]:
df = generic_info()

In [ ]:
dfUser = userInfo()

In [ ]:
result = pd.concat([df,dfUser],axis=1)

#### API.get_user(id/user_id/screen_name): to get all the user info

In [ ]:
#@params passing tweet, friendlist and userinfo(in case of following)
#returns data frame
def getTweetinfo(tweetObj,friendList,user=None):
    if user == None:
        data = pd.DataFrame(
          {
             'tweetId':tweetObj.id_str,
             'userID':tweetObj.user.id,
             'parentID':'None',
             'favourites_count':tweetObj.user.favourites_count,
             'userName':tweetObj.user.name,
             'userDescription':tweetObj.user.description,
             'userCreatedAt':tweetObj.user.created_at,
             'imageurl':tweetObj.user.profile_image_url,
             'userFollowersCount':tweetObj.user.followers_count,
             'friendsCount':tweetObj.user.friends_count,
             'friendList':[friendList] 
          })
    else:
        data = pd.DataFrame(
        {
             'tweetId': "None",
             'userID':user.id,
             'parentID': tweetObj.user.id,
             'favourites_count':"None",
             'userName': user.name,
             'userDescription':user.description,
             'userCreatedAt':user.created_at,
             'imageurl':user.profile_image_url,
             'userFollowersCount':user.followers_count,
             'friendsCount':user.friends_count,
             'friendList': "None"
        },index=[0])
    return data

In [ ]:
# function to get twitter and user info and return df
#@params api: api_handler, hashtags: query parameters, inceptionDate: start date for hashtags, lang: for language restriction 
def getTweetInfo(api,queryParams,inceptionDate,lang="en"):
    df = pd.DataFrame([])
#     df.astype('object')       # as some data contains list
    for tweet in tweepy.Cursor(api.search,q= (queryParams),since_id=inceptionDate,lang=lang).items(2): # (lang=en) : for language restriction # items value set only for test
        friendList = [friend for friend in tweepy.Cursor(api.friends_ids, id= tweet.user.id).items(5)]  # items value set only for test
        data = getTweetinfo(tweet,friendList=friendList)
        df = df.append(data,ignore_index=True,sort=False)
        for userID in friendList:
            user = api.get_user(userID)
            userData = getTweetinfo(tweet,friendList=friendList,user=user)
            if user.id not in df.userID:  #prevent duplication in data
                df = df.append(userData,ignore_index=True,sort=False)
    return df

In [ ]:
hashtags = '#juulvapor OR #juulnation OR #doit4juul OR #juul'
lang = "en"
inceptionDate = "2018-08-08"

df = getTweetInfo(api,hashtags,inceptionDate)

In [ ]:
writer = pd.ExcelWriter('juul.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')